# 1. Get data from Azure SQL DB

In [ ]:
import pyodbc

server = 'dataproject.database.windows.net'
database = 'Data515Project'
username = 'uberandtaxinyc'
password = 'Lzcnhwzcylyw515!'
driver= '{ODBC Driver 13 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';PORT=1433;SERVER='+server+';PORT=1443;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute("SELECT COUNT(*) FROM [dbo].[apr_time] WHERE [day] = 14")
row = cursor.fetchone()
while row:
    print(row)
    row = cursor.fetchone()

# 2. Import and clean the NYC shape data

In [1]:
import json

def read_json(filename):
    """
    A function read the json file and process the Neighborhoods and Coordinates

    Parameter: filename, a json file)

    Return: result, a dictionary contains
            key: Neighborhood,
            value: a list of coordinate corresponding to the that neighborhood
    """

    with open(filename) as f:
        data = json.load(f)

    result = {}
    for feature in data['features']:
        
        # NATName = Neighborhood Tabulation Area Name
        
        neighborhood = feature['properties']['NTAName']
        coordinates = feature['geometry']['coordinates'][0] # 3 layers or 2 layers
        
        if len(coordinates[0]) > 2:
            # 3 layers
            coordinates = coordinates[0]

        result[neighborhood] = coordinates

        
    return result


In [2]:
shapes = read_json("./data/NYC_Shapes.json")

In [3]:
# result = {"Borough Park" : [[lat1, lon1], [lat2, lon2], ...]}

"""
polygons =
{
"Borough Park" : {Lat : [], Lon : []}
"East Flushing" : {Lat : [], Lon : []}
"Auburndale" : {Lat : [], Lon : []}
.
.
.
"Elmhurst" : {Lat : [], Lon : []}
}
"""

def process_coordinates(result):
    """
    A function read the dictionary contains

    key: neighborhood
    value: list of coordinates (latitude, longitude)

    and reconstruct a new dictionary contains
    key: neighborhood
    value: a dictionary contains a list of latitudes and a list of longitudes.

    Parameter: result dictionary, contains neighborhoods and list of coordinates

    Return: polygon dictionary, contains neighborhoods
            and a list of latitudes and a list of longitudes
    """

    polygons = {}
    for neighborhood in shapes.keys():

        coordinates = result[neighborhood]

        lat_list = []
        lon_list = []

        for coordinate in coordinates:
            lat_list.append(coordinate[1])
            lon_list.append(coordinate[0])
        polygons[neighborhood] = {}
        polygons[neighborhood]["Lat"] = lat_list
        polygons[neighborhood]["Lon"] = lon_list

    return polygons

In [4]:
nyc_neighborhoods = process_coordinates(shapes)

In [ ]:
len(nyc_neighborhoods)

# 3. Build Bokeh heat map

In [5]:
import numpy as np
import pandas as pd

import bokeh.layouts
from bokeh.io import output_file, show
from bokeh.layouts import widgetbox
from bokeh.models import (
    BoxSelectTool,
    Circle, 
    ColumnDataSource,
    CustomJS,
    DataRange1d, 
    GMapPlot,
    GMapOptions,
    HoverTool,
    LogColorMapper,
    PanTool,
    Slider,
    WheelZoomTool,
)
from bokeh.models.widgets import Button, RadioButtonGroup, Select, CheckboxButtonGroup
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.models.glyphs import Patches

In [6]:
palette.reverse()

neighborhood_xs = [neighborhood["Lon"] for neighborhood in nyc_neighborhoods.values()]
neighborhood_ys = [neighborhood["Lat"] for neighborhood in nyc_neighborhoods.values()]
neighborhoods = list(nyc_neighborhoods.keys())
num_nodes = [len(neighborhood["Lat"]) for neighborhood in nyc_neighborhoods.values()]

color_mapper = LogColorMapper(palette=palette)

source = ColumnDataSource(data=dict(
    x=neighborhood_xs,
    y=neighborhood_ys,
    neighborhood=neighborhoods,
    node=num_nodes,
))

# Interactions
TOOLS = "pan,wheel_zoom,reset,hover,save"

plot = figure(title="Uber Pickup Distribution", tools=TOOLS, x_axis_location=None, y_axis_location=None
)
plot.grid.grid_line_color = None

plot.patches('x', 'y', source=source,
              fill_color={'field': 'node', 'transform': color_mapper},
              fill_alpha=0.7, line_color="black", line_width=0.5)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data
    var f = cb_obj.value;
    node = data['node']
    for (i = 0; i < node.length; i++) {
        node[i] = node[i] * f / 2
    }
    source.trigger('change');
""")

hour_slider = Slider(start=0, end=23, step=1, title="Hour", callback = callback)
hour_slider.js_on_change('start', callback)


In [7]:
hover = plot.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@neighborhood"),
    ("Number of nodes", "@node"),
    ("(Long, Lat)", "($x, $y)"),
]

checkbox_button_group = CheckboxButtonGroup(labels=["Uber", "Taxi"], active=[0, 1])

select1 = Select(title="Month:", value="Month", options=["April", "May", "June", "July", "August", "September"])
select2 = Select(title="Date:", value="Date", options=["1", "2", "3", "4", "5", "6", "7"])


In [8]:
show(bokeh.layouts.row(plot, checkbox_button_group, widgetbox(hour_slider, select1, select2, width=300)))